##### Copyright 2021 Google Inc.

Licensed under the Apache License, Version 2.0 (the "License").
<!--
    Licensed to the Apache Software Foundation (ASF) under one
    or more contributor license agreements.  See the NOTICE file
    distributed with this work for additional information
    regarding copyright ownership.  The ASF licenses this file
    to you under the Apache License, Version 2.0 (the
    "License"); you may not use this file except in compliance
    with the License.  You may obtain a copy of the License at

      http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing,
    software distributed under the License is distributed on an
    "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
    KIND, either express or implied.  See the License for the
    specific language governing permissions and limitations
    under the License.
-->


# Beam SQL in notebooks

This example demonstrates how to use SQL to write Apache Beam pipelines, use the `beam_sql` magic (since Beam v2.34.0) to quickly iterate the pipeline development in notebooks, and launch Dataflow jobs with Beam SQL from notebooks.

## An overview of Beam SQL

[Beam SQL](https://beam.apache.org/documentation/dsls/sql/overview/) allows a Beam user (currently only available in Beam Java and Python) to query bounded and unbounded PCollections with SQL statements. Your SQL query is translated to a PTransform, an encapsulated segment of a Beam pipeline. You can freely mix SQL PTransforms and other PTransforms in your pipeline.

Since SQL support in Beam Python SDK is implemented through xLang external transform, make sure you have below prerequisites:
- Have `docker` installed;
- Have jdk8 or jdk11 installed.

In [ ]:
# The notebook environment should have docker and jdk 1.8 installed.
!docker image list
!java -version

In [ ]:
# Optionally sets the logging level to reduce distraction.
import logging

logging.root.setLevel(logging.ERROR)

**Important**: If you're using Beam built from your local source code, additionally:

- Have the Java expansion service shadowjar built. Go to the root directory of your local beam repo and then execute:
  `./gradlew :sdks:java:extensions:sql:expansion-service:shadowJar`.
- Based on your jdk version, pull the docker image `docker pull apache/beam_java11_sdk` or `docker pull apache/beam_java8_sdk`.
- Then tag the image with your current Beam dev version.  You can check the dev version under `apache_beam.version.__version__`. For example, if you're using jdk11 and dev version is `x.x.x.dev`, execute `docker image tag apache/beam_java11_sdk:latest apache/beam_java11_sdk:x.x.x.dev`.

### Run Beam SQL in notebooks with `beam_sql` magic

[Beam SQL](https://beam.apache.org/documentation/dsls/sql/overview/) allows a Beam user to query PCollections with SQL statements. Currently, `InteractiveRunner` does not support `SqlTransform` due to [BEAM-10708](https://issues.apache.org/jira/browse/BEAM-10708). However, a user could use the `beam_sql` magic to run Beam SQL in the notebook and introspect the result. 

`beam_sql` is an IPython [custom magic](https://ipython.readthedocs.io/en/stable/config/custommagics.html). If you're not familiar with magics, here are some [built-in examples](https://ipython.readthedocs.io/en/stable/interactive/magics.html). It's a convenient way to validate your queries locally against known/test data sources when prototyping a Beam pipeline with SQL, before productionizing it on remote cluster/services.

The notebook environment has preloaded the `beam_sql` magic. You can also explicitly load it via `%load_ext apache_beam.runners.interactive.sql.beam_sql_magics` if you set up your own notebook elsewhere.

**Note**: `beam_sql` is for `apache_beam[interactive]>=2.34.0`. Make sure to connect your notebook to a kernel that meets the requirement.

The `beam_sql` magic can be used as either a line magic or a cell magic. You can check its usage by running:

In [ ]:
%beam_sql -h

The advantages of using `beam_sql` in notebooks are:
- all scenarios use the same intuitive syntax and you don't have to differentiate them from each other any more
    - no need to use the constant `PCOLLECTION` when querying a single PCollection
    - no need to name multiple input PCollections, instead referring them by their variable names
- no need to write SqlTransform and other Beam related boilerplates
- can introspect the result immediately without running the pipeline explicitly or implicitily through a context manager
- automatically handles coder registration for your PCollection schemas

### There are three scenarios for Beam SQL

When writing Beam SQL, this notebook uses the `beam_sql` magic because of its advantages over the `SqlTransform`. *A comparison between using the `beam_sql` magic and `SqlTransform` to write Beam SQL can be found in the [appendix](#Beam-SQL-without-beam_sql-magic)*.

#### 1. Use Beam SQL to create a PCollection from constant values

In [ ]:
%%beam_sql -o pcoll
SELECT CAST(1 AS INT) AS `id`, CAST('foo' AS VARCHAR) AS `str`, CAST(3.14 AS DOUBLE) AS `flt`

#### 2. Use Beam SQL to query a PCollection

In [ ]:
%%beam_sql -o id_pcoll
SELECT id FROM pcoll

#### 3. Use Beam SQL to query multiple PCollections (joined)

In [ ]:
%%beam_sql -o value_with_same_id
SELECT * FROM pcoll JOIN id_pcoll USING (id)

## Example queries

Now that you are familiar with the Beam SQL and the `beam_sql` magic, let's view a few more examples with advanced usage of the `beam_sql` magic such as mixing it with other Beam I/O connectors.

### Query#1 - A simple static query

You can run a simple SQL query (in Apache Calcite SQL [syntax](https://beam.apache.org/documentation/dsls/sql/calcite/query-syntax/)) to create a [schema-aware PCollection](https://beam.apache.org/documentation/programming-guide/#schemas) from constant values.

In [ ]:
%%beam_sql -o query1_data
SELECT CAST(5 AS INT) AS `id`, CAST('foo' AS VARCHAR) AS `str`, CAST(3.14 AS DOUBLE) AS `flt`

The `beam_sql` magic shows you the result of the SQL query.

It also creates and outputs a PCollection named `query1_data` with `element_type` like `BeamSchema_...(id: int32, str: str)`.

Note that you have **not** explicitly created a Beam pipeline. You get a PCollection because the `beam_sql` magic always **implicitly creates** a pipeline to execute your SQL query. To hold the elements with each field's type info, Beam automatically creates a schema as the `element_type` for the created PCollection.

To introspect the data again with more knobs, you can use `show`.

In [ ]:
from apache_beam.runners.interactive import interactive_beam as ib
ib.show(query1_data)
# Uncomment below to set more args.
# ib.show(query1_data, visualize_data=True, include_window_info=True)

To materialize the PCollection into a pandas [DataFrame](https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html#dataframe) object, you can use `collect`.

In [ ]:
ib.collect(query1_data)

You can also additionally append some transforms such as writing to a text file and print the elements:

In [ ]:
import apache_beam as beam

coder=beam.coders.registry.get_coder(query1_data.element_type)
print(coder)
query1_data | beam.io.textio.WriteToText('/tmp/query1_data', coder=coder)
query1_data | beam.Map(print)

Execute the pipeline as a normal pipeline running on DirectRunner and inspect the output file.

In [ ]:
!rm -rf /tmp/query1_data*
query1_data.pipeline.run().wait_until_finish()
!ls /tmp/query1_data*
!cat /tmp/query1_data*

The coder in use is a `RowCoder`. The element is encoded and written to the text file. When inspecting it directly, it may display **garbled strings**. The file will be revisited later in Query#4.

### [Optional] Omit the `-o` option.
If the option is omitted, an output name is auto-generated based on the SQL query and PCollection (if any) it queries. Optionally, you can also use the `_[{execution_count}]` convention: `_` for last output and `_{execution_count}` for a specific cell execution output.

However, explicitly naming the output is recommended for better notebook readability and to avoid unexpected errors.

Below example outputs a PCollection named like `sql_output_...`.

In [ ]:
%%beam_sql
SELECT CAST(1 AS INT) AS `id`, CAST('foo' AS VARCHAR) AS `str`, CAST(3.14 AS DOUBLE) AS `flt`

Now that you are familiar with the `beam_sql` magic, you can build more queries against PCollections.

Let's install the `names` package to randomly generate some names.

In [ ]:
%pip install names

Import all modules needed for this example.

In [ ]:
import names
import typing

import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
from apache_beam.runners.interactive import interactive_beam as ib

Create a pipeline `p` with the `InteractiveRunner`.

In [ ]:
p = beam.Pipeline(InteractiveRunner())

Then let's create a schema with `typing.NamedTuple`. Let's call it `Person` with a field `id` and a field `name`.

In [ ]:
class Person(typing.NamedTuple):
    id: int
    name: str

With `beam_sql` magic, you can utilize all the Beam I/O connectors (streaming is currently not supported due to `DirectRunner` not supporting streaming pipeline with `SqlTransform`) as source of data, then build a SQL query against all the data and check the output. If needed, you can sink the output following the `WriteToText` example demonstrated above.

## Query#2 - Querying a single PCollection

Let's build a PCollection with 10 random `Person` typed elements.

In [ ]:
persons = (p 
           | beam.Create([Person(id=x, name=names.get_full_name()) for x in range(10)]))
ib.show(persons)

You can look for all elements with `id < 5` in `persons` with the below query and assign the output to `persons_id_lt_5`. Also, you can enable `-v` option to see more details about the execution.

In [ ]:
%%beam_sql -o persons_id_lt_5 -v
SELECT * FROM persons WHERE id <5

With `-v`, if it's the first time running this query, you might see a warning message about

```
Schema Person has not been registered to use a RowCoder. Automatically registering it by running: beam.coders.registry.register_coder(Person, beam.coders.RowCoder)
```

The `beam_sql` magic helps registering a `RowCoder` for each schema you define and use whenever it finds one. You can also explicitly run the same code to do so.

Note the output element type is `Person(id: int, name: str)` instead of `BeamSchema_...` because you have selected all the fields from a single PCollection of the known type `Person(id: int, name: str)`.

## Query#3 - Joining multiple PCollections

You can build a `persons_2` PCollection with a different range of `id`s and `name`s. 

In [ ]:
persons_2 = (p 
             | beam.Create([Person(id=x, name=names.get_full_name()) for x in range(5, 15)]))
ib.show(persons_2)

Then query for all `name`s from `persons` and `persons_2` with the same `id`s and assign the output to `persons_with_common_id`.

In [ ]:
%%beam_sql -o persons_with_common_id -v
SELECT * FROM persons JOIN persons_2 USING (id)

Note the output element type is now some `BeamSchema_...(id: int64, name: str, name0: str)`. Because you have selected columns from both PCollections, there is no known schema to hold the result. Beam automatically creates a schema and differentiates conflicted field `name` by suffixing `0` to one of them.

And since `Person` is already previously registered with a `RowCoder`, there is no more warning about registering it anymore even with `-v`.

## Query#4 - Join multiple PCollections, including I/O.

Let's read the file written by Query#1 and use it to join `persons` and `persons_2` to find `name`s with the common `id` in all three of them. 

In [ ]:
# Use the exact same coder used when WriteToText and explicitly set the output types.
query1_result_in_file = p | beam.io.ReadFromText(
    '/tmp/query1_data*', coder=coder).with_output_types(
    query1_data.element_type)

# Check all the data sources.
ib.show(query1_result_in_file)
ib.show(persons)
ib.show(persons_2)

In [ ]:
%%beam_sql -o entry_with_common_id

SELECT query1_result_in_file.id, persons.name AS `name_1`, persons_2.name AS `name_2`
FROM query1_result_in_file JOIN persons ON query1_result_in_file.id = persons.id
JOIN persons_2 ON query1_result_in_file.id = persons_2.id

You can also chain another `beam_sql` magic to get just `name_1`:

In [ ]:
%%beam_sql -o name_found
SELECT name_1 AS `name` FROM entry_with_common_id

## Run Beam SQL on Dataflow via `beam_sql` magic

Next you can execute the same SQL to pick only `name_1` column from the `entry_with_common_id` but run it on Dataflow by specifying `-r DataflowRunner`.

A form will be generated below for you to fill in minimum pipeline options needed. Some of the fields might have been auto-populated based on the context of this notebook environment.

There are 2 buttons:
- `Run on Dataflow` submits a Dataflow job from this notebook.
- `Show Options` shows you the current pipeline options configured for the job to be submitted.

**Important**: If you're using Beam built from source code, you need to execute the cell after next cell to set `sdk_location` before clicking the `RUN ON DATAFLOW` button generated by this cell.

**Tips**: In the form generated by the `beam_sql` magic, use `gs://your-GCS-bucket` as the `GCS Bucket` and put `names` in the `Additional Packages`. The output PCollection will be automatically saved to `gs://your-GCS-bucket/staging/on_dataflow` file on Cloud Storage.

In [ ]:
%%beam_sql -o on_dataflow -r DataflowRunner
SELECT name_1 AS `name` FROM entry_with_common_id

In [ ]:
# Uncomment and execute iff you're using Beam built from source code.
# from apache_beam.options.pipeline_options import SetupOptions
# options_on_dataflow.view_as(SetupOptions).sdk_location = '/dir/to/your/apache-beam-x.xx.x.tar.gz'

In [ ]:
# Replace your-GCS-bucket with the real input and execute once the dataflow job is done.
!gsutil cat 'gs://your-GCS-bucket/staging/abc-00000-of-00001'

## What's the difference between pipelines using Beam SQL and those not using it?

Let's build a pipeline to find out the data for the state with the most COVID positive cases on a specific day.
### A pipeline without using Beam SQL

#### Get the data

In [ ]:
import json
import requests

# The covidtracking project has stopped collecting new data, current data ends on 2021-03-07
json_current='https://covidtracking.com/api/v1/states/current.json'

def get_json_data(url):
  with requests.Session() as session:
    data = json.loads(session.get(url).text)
  return data

current_data = get_json_data(json_current)

#### Create a PCollection from the data

In [ ]:
p = beam.Pipeline(runner=InteractiveRunner())
raw_data = p | 'Create PCollection from json' >> beam.Create(current_data)

#### Create a orderable wrapper for the data

In [ ]:
from functools import total_ordering

@total_ordering
class UsCovidDataOrderByPositive:
    def __init__(self, data):
        self.data = data
    
    def __gt__(self, other):
        return self.data['positive'] > other.data['positive']

#### Pick 4 columns from the data

Below code uses a plain dictionary to hold the data.

You don't have to define a schema nor explicitily be aware of the type of each field. 

But this can be dangerous when you write a more complicated pipeline where your assumptions about the data is incorrect.

With `visualize_data=True` in `show()`, you can spot the element with the maximum positive in the visualization by setting `Color By positive`.

In [ ]:
data = raw_data | 'Parse' >> beam.Map(
    lambda e: {
        'date': e['date'], 
        'state': e['state'], 
        'positive': e['positive'], 
        'negative': e['negative']})
ib.show(data, visualize_data=True)

Find the element with the most positive.

This is rather verbose, you have to wrap the data into an orderable wrapper, find the maximum entry by the ordering and then unwrap the data.

In [ ]:
entry_with_max_positive = (
    data | 'Data OrderByPositive' >> beam.Map(lambda e: UsCovidDataOrderByPositive(e))
         | 'Find Maximum Positive' >> beam.CombineGlobally(max)
         | 'Convert Back to Data' >> beam.Map(lambda orderable_data: orderable_data.data))
ib.show(entry_with_max_positive)

### A pipeline using Beam SQL

Let's build the same pipeline but with Beam SQL.

There is something wrong with the below schema. It works with normal Beam usage in Python but it doesn't work with Beam SQL. Can you spot the mistakes?

In [ ]:
from typing import NamedTuple


class UsCovidData(NamedTuple):
    date: str
    state: str
    positive: int
    negative: int

The answer:

- `date` is a keyword in (Calcite)SQL, use a different field name such as `partition_date`;
- `date` from the data is an integer type, not str. Make sure you convert the data using `str()` or use `date: int`.
- `negative` has missing values and the default is None. So instead of `negative: int`, it should be `negative: Optional[int]`. Or you can convert None into 0 when using the schema.

In [ ]:
from typing import Optional


# Adjusted schema based on the data
class UsCovidData(NamedTuple):
    partition_date: str  # Remember to str(e['date']).
    state: str
    positive: int
    negative: Optional[int]

#### Read data

In [ ]:
p_sql = beam.Pipeline(runner=InteractiveRunner())
covid_data = (p_sql 
        | 'Create PCollection from json' >> beam.Create(current_data)
        | 'Parse' >> beam.Map(
            lambda e: UsCovidData(
                partition_date=str(e['date']),
                state=e['state'],
                positive=e['positive'],
                negative=e['negative'])).with_output_types(UsCovidData))
ib.show(covid_data)

#### Find the maximum positive value

In [ ]:
%%beam_sql -o max_positive
SELECT partition_date, MAX(positive) AS `positive`
FROM covid_data
GROUP BY partition_date

#### Join the maximum positive value with the original data to get the rest fields.

Below code also handles the `negative is None` case to use a default value 0.

In [ ]:
%%beam_sql -o entry_with_max_positive
SELECT covid_data.partition_date, covid_data.state, covid_data.positive, {fn IFNULL(covid_data.negative, 0)} AS `negative`
FROM covid_data JOIN max_positive
USING (partition_date, positive)

## Appendix

### Beam SQL without `beam_sql` magic

#### 1. Use Beam SQL to create a PCollection from constant values

Note the `SqlTransform` is applied to a pipeline not a PCollection, unlike the below 2 scenarios.

In [ ]:
from apache_beam.transforms.sql import SqlTransform


with beam.Pipeline() as p:
    pcoll = p | 'Create pcoll' >> SqlTransform("""
        SELECT CAST(1 AS INT) AS `id`, CAST('foo' AS VARCHAR) AS `str`, CAST(3.14 AS DOUBLE) AS `flt`""")
    _ = pcoll | beam.io.WriteToText('/tmp/pcoll')

!cat /tmp/pcoll*

#### 2. Use Beam SQL to query a PCollection

You have to use the `PCOLLECTION` constant in the query.

Below query selects `id` field from the above `pcoll`.

In [ ]:
with beam.Pipeline() as p:
    pcoll = p | 'Create pcoll' >> SqlTransform("""
        SELECT CAST(1 AS INT) AS `id`, CAST('foo' AS VARCHAR) AS `str`, CAST(3.14 AS DOUBLE) AS `flt`""")
    id_pcoll = pcoll | 'Select id from pcoll' >> SqlTransform("""SELECT id FROM PCOLLECTION""")
    _ = id_pcoll | beam.io.WriteToText('/tmp/id')

!cat /tmp/id*

#### 3. Use Beam SQL to query multiple PCollections (joined)
You can tag PCollections with names and refer them in the query using the tagged names.

Below query joins previous 2 PCollections by `id`, should return the original element from `pcoll`.

In [ ]:
with beam.Pipeline() as p:
    pcoll = p | 'Create pcoll' >> SqlTransform("""
        SELECT CAST(1 AS INT) AS `id`, CAST('foo' AS VARCHAR) AS `str`, CAST(3.14 AS DOUBLE) AS `flt`""")
    id_pcoll = pcoll | 'Select id from pcoll' >> SqlTransform("""SELECT id FROM PCOLLECTION""")
    value_with_same_id = {'input_1': pcoll, 'input_2': id_pcoll} | 'Join pcolls' >> SqlTransform("""
        SELECT * FROM input_1 JOIN input_2 USING (id)""")
    _ = value_with_same_id | beam.io.WriteToText('/tmp/same_id')

!cat /tmp/same_id*